In [22]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import warnings
from collections import Counter
import yfinance as yf
import os
import backtrader as bt
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical
warnings.filterwarnings('ignore', category=FutureWarning)


In [23]:
import yfinance as yf

tickers = ['AAPL']  # Corrected ticker symbol for Apple Inc.
start = '2023-01-01'  # Corrected date format
end = '2023-08-01'  # Corrected date format
interval = '1h'

df = yf.download(tickers, start=start, end=end, threads=True, interval=interval)


[*********************100%***********************]  1 of 1 completed


In [24]:
df.tail(2)

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-07-31 14:30:00-04:00,196.175003,196.350006,195.934998,196.087601,196.087601,3009785
2023-07-31 15:30:00-04:00,196.080002,196.490005,195.789993,196.490005,196.490005,5194547


In [25]:
# class BollingerBandsStrategy(bt.Strategy):
#     params = (
#         ('period', 20),       # Number of periods for the moving average
#         ('devfactor', 2),     # Number of standard deviations for the bands
#     )

#     def __init__(self):
#         self.bollinger = bt.indicators.BollingerBands(self.data.close, period=self.p.period,
#                                                       devfactor=self.p.devfactor)
#         self.dist_upper = self.data.close - self.bollinger.lines.bot
#         self.dist_lower = self.bollinger.lines.top - self.data.close

#     def next(self):
#         self.log('Close: {:.2f}, Upper: {:.2f}, Lower: {:.2f}, Distance Upper: {:.2f}, Distance Lower: {:.2f}'
#                  .format(self.data.close[0], self.bollinger.lines.top[0], self.bollinger.lines.bot[0],
#                          self.dist_upper[0], self.dist_lower[0]))

#         # Add your buy/sell/do-nothing logic based on the distances here
#         # For example:
#         if self.dist_upper[0] > 0:
#             self.sell()
#         elif self.dist_lower[0] > 0:
#             self.buy()

# Custom Backtrader indicator for Bollinger Bands calculation
class BollingerBands(bt.Indicator):
    lines = ('top', 'mid', 'bot', 'dist_upper', 'dist_lower', 'dist_upper_signal', 'dist_lower_signal', 'buy_signal')

    params = (
        ('period', 20),  # Period for the moving average
        ('devfactor', 2.0),  # Number of standard deviations for the bands
        ('dist_upper_signal', -1),  # Value for upper band signal
        ('dist_lower_signal', 1),  # Value for lower band signal
        ('buy_signal', 0),  # Value for buy signal
    )

    def __init__(self):
        self.l.mid = bt.indicators.SMA(self.data, period=self.p.period)
        self.l.top = self.l.mid + self.p.devfactor * bt.indicators.StandardDeviation(self.data, period=self.p.period)
        self.l.bot = self.l.mid - self.p.devfactor * bt.indicators.StandardDeviation(self.data, period=self.p.period)

        self.l.dist_upper = self.data - self.l.top
        self.l.dist_lower = self.data - self.l.bot
        self.l.dist_upper_signal = bt.If(self.data > self.l.top, self.p.dist_upper_signal, 0)
        self.l.dist_lower_signal = bt.If(self.data < self.l.bot, self.p.dist_lower_signal, 0)
        self.l.buy_signal = bt.If(self.l.dist_upper > 0, self.p.buy_signal, 0)

In [ ]:
# class BollingerBandsStrategy(bt.Strategy):
#     params = (
#         ('period', 20),       # Number of periods for the moving average
#         ('devfactor', 2),     # Number of standard deviations for the bands
#     )

#     def __init__(self):
#         self.bollinger = bt.indicators.BollingerBands(self.data.close, period=self.p.period,
#                                                       devfactor=self.p.devfactor)
#         self.dist_upper = self.data.close - self.bollinger.lines.bot
#         self.dist_lower = self.bollinger.lines.top - self.data.close

#     def next(self):
#         self.log('Close: {:.2f}, Upper: {:.2f}, Lower: {:.2f}, Distance Upper: {:.2f}, Distance Lower: {:.2f}'
#                  .format(self.data.close[0], self.bollinger.lines.top[0], self.bollinger.lines.bot[0],
#                          self.dist_upper[0], self.dist_lower[0]))

#         # Add your buy/sell/do-nothing logic based on the distances here
#         # For example:
#         if self.dist_upper[0] > 0:
#             self.sell()
#         elif self.dist_lower[0] > 0:
#             self.buy()

# Custom Backtrader indicator for Bollinger Bands calculation
class BollingerBands(bt.Indicator):
    lines = ('top', 'mid', 'bot', 'dist_upper', 'dist_lower', 'dist_upper_signal', 'dist_lower_signal', 'buy_signal')

    params = (
        ('period', 20),  # Period for the moving average
        ('devfactor', 2.0),  # Number of standard deviations for the bands
        ('dist_upper_signal', -1),  # Value for upper band signal
        ('dist_lower_signal', 1),  # Value for lower band signal
        ('buy_signal', 0),  # Value for buy signal
    )

    def __init__(self):
        self.l.mid = bt.indicators.SMA(self.data, period=self.p.period)
        self.l.top = self.l.mid + self.p.devfactor * bt.indicators.StandardDeviation(self.data, period=self.p.period)
        self.l.bot = self.l.mid - self.p.devfactor * bt.indicators.StandardDeviation(self.data, period=self.p.period)

        self.l.dist_upper = self.data - self.l.top
        self.l.dist_lower = self.data - self.l.bot
        self.l.dist_upper_signal = bt.If(self.data > self.l.top, self.p.dist_upper_signal, 0)
        self.l.dist_lower_signal = bt.If(self.data < self.l.bot, self.p.dist_lower_signal, 0)
        self.l.buy_signal = bt.If(self.l.dist_upper > 0, self.p.buy_signal, 0)

In [80]:
class MACD:
    def __init__(self, data):
        # Calculate MACD using default settings (12, 26, 9)
        self.macd, self.signal, self.hist = self.calculate_macd(data)

    def calculate_macd(self, data):
        # Implement the logic to calculate MACD here (you can use Pandas or Numpy)
        # Replace the code below with your actual MACD calculation
        close_prices = data['Close']
        ema_12 = close_prices.rolling(window=12, min_periods=1).mean()
        ema_26 = close_prices.rolling(window=26, min_periods=1).mean()
        macd = ema_12 - ema_26
        signal = macd.rolling(window=9, min_periods=1).mean()
        hist = macd - signal
        return macd, signal, hist

# Custom Backtrader indicator for MACD calculation
# class MACD(bt.Indicator):
#     lines = ('macd', 'signal', 'hist')

#     params = (
#         ('period_me1', 12),
#         ('period_me2', 26),
#         ('period_signal', 9),
#     )

#     def __init__(self):
#         me1 = bt.indicators.EMA(self.data, period=self.p.period_me1)
#         me2 = bt.indicators.EMA(self.data, period=self.p.period_me2)
#         self.l.macd = me1 - me2
#         self.l.signal = bt.indicators.EMA(self.l.macd, period=self.p.period_signal)
#         self.l.hist = self.l.macd - self.l.signal

In [27]:
def calculate_bollinger_bands_signal(data):
    # Implement the logic to calculate Bollinger Bands signal here
    # Replace the code below with your actual Bollinger Bands calculation
    close_prices = data['Close']
    sma = close_prices.rolling(window=20, min_periods=1).mean()
    std = close_prices.rolling(window=20, min_periods=1).std()
    upper_band = sma + 2 * std
    lower_band = sma - 2 * std
    signal = np.where(close_prices > upper_band, -1, np.where(close_prices < lower_band, 1, 0))
    return signal 

In [87]:
def prepare_data(data, lookback):
    # Calculate MACD using custom class (you can replace this with your own MACD implementation)
    macd = MACD(data)

    # Calculate Bollinger Bands signals (you can replace this with your own Bollinger Bands implementation)
    bollinger_signal = calculate_bollinger_bands_signal(data)

    # Combine the features (MACD and Bollinger Bands signals) into a single DataFrame
    feature_df = pd.DataFrame({
        'macd': macd.macd,
        'signal': macd.signal,
        'hist': macd.hist,
        'bollinger_signal': bollinger_signal
    })

    # Calculate Bollinger Bands distances (dist1, dist2, dist3)
    feature_df['dist1'] = data['Close'] - feature_df['bollinger_signal'].rolling(window=20).mean()
    feature_df['dist2'] = data['Close'] - feature_df['bollinger_signal'].rolling(window=50).mean()
    feature_df['dist3'] = data['Close'] - feature_df['bollinger_signal'].rolling(window=200).mean()

    # Drop NaN values (due to the rolling window) and reset the index
    feature_df = feature_df.dropna().reset_index(drop=True)

    # Prepare the input data and target labels
    X = []
    y = []
    for i in range(len(feature_df) - lookback):
        # Extract the features for the current sequence
        inputs = feature_df.loc[i : i + lookback - 1].values

        # Extract the target label for the next time step
        target = feature_df.loc[i + lookback, 'bollinger_signal']

        X.append(inputs)
        y.append(target)

    # Convert the lists to Numpy arrays
    X_train = np.array(X)
    y_train = np.array(y)

    # Normalize the input data (you can choose your own normalization method)
    X_train = (X_train - X_train.mean()) / X_train.std()

    return X_train, y_train


In [88]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [89]:
def create_rnn_model():
    model = Sequential()
    model.add(LSTM(32))
    # model.add(Dropout(0.2))
    model.add(Dense(16, activation='softmax'))
    model.add(Dense(8, activation='softmax'))
    model.add(Dense(3, activation='softmax'))
    return model

In [90]:
def train_rnn_model(model, X_train, y_train, epochs=10, batch_size=32):
    # Convert target labels to one-hot encoding for multi-class classification
    y_train_categorical = to_categorical(y_train, num_classes=3)
    print(y_train_categorical)

    # Replace this function with the code to train your RNN model using Keras with teacher forcing
    # Use your preferred loss function, optimizer, and other hyperparameters
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train_categorical, epochs=epochs, batch_size=batch_size)
    return model

In [92]:
input_data, target_labels = prepare_data(df, lookback=30) 

In [93]:
model = create_rnn_model() 

In [94]:
model = train_rnn_model(model, input_data, target_labels, 50, 32) 

[[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
Epoch 1/50


25/25 [==============================] - 3s 7ms/step - loss: 0.9681 - accuracy: 0.8994
Epoch 2/50
25/25 [==============================] - 0s 7ms/step - loss: 0.9024 - accuracy: 0.8994
Epoch 3/50
25/25 [==============================] - 0s 7ms/step - loss: 0.8469 - accuracy: 0.8994
Epoch 4/50
25/25 [==============================] - 0s 7ms/step - loss: 0.7993 - accuracy: 0.8994
Epoch 5/50
25/25 [==============================] - 0s 7ms/step - loss: 0.7552 - accuracy: 0.8994
Epoch 6/50
25/25 [==============================] - 0s 7ms/step - loss: 0.7159 - accuracy: 0.8994
Epoch 7/50
25/25 [==============================] - 0s 7ms/step - loss: 0.6806 - accuracy: 0.8994
Epoch 8/50
25/25 [==============================] - 0s 8ms/step - loss: 0.6487 - accuracy: 0.8994
Epoch 9/50
25/25 [==============================] - 0s 7ms/step - loss: 0.6204 - accuracy: 0.8994
Epoch 10/50
25/25 [==============================] - 0s 7ms/step - loss: 0.5952 - accuracy: 0.8994
Epoch 11/50
25/25 [===========

In [95]:
import yfinance as yf
import numpy as np
import pandas as pd
from tensorflow import keras
import backtrader as bt

# Define the custom Deep Learning RNN strategy using Backtrader
class DL_RNN_Strategy(bt.Strategy):
    params = (
        ('lookback', 30),  # Number of historical data points to use as input
        ('buy_threshold', 0),  # Threshold for buy signal (0 means buy when positive dist1)
        ('sell_threshold', 0),  # Threshold for sell signal (0 means sell when negative dist3)
    )

    def __init__(self, model):
        self.macd = MACD(self.data)  # Calculate MACD using custom indicator
        self.bollinger = BollingerBands(self.data)  # Calculate Bollinger Bands using custom indicator
        self.signals = []
        self.model = model

    def next(self):
        if len(self.data) >= self.p.lookback:
            # Calculate Bollinger Bands signals and distances
            bollinger_signal = self.bollinger.lines.dist_upper.get(ago=self.p.lookback)
            dist1 = self.bollinger.lines.dist_upper.get(size=self.p.lookback)
            dist2 = self.bollinger.lines.dist_lower.get(size=self.p.lookback)
            dist3 = self.bollinger.lines.dist_upper.get(size=self.p.lookback)

            # Extract MACD values from the custom indicator
            macd_values = self.macd.lines.macd.get(size=self.p.lookback)
            signal_values = self.macd.lines.signal.get(size=self.p.lookback)
            hist_values = self.macd.lines.hist.get(size=self.p.lookback)

            # Print the sizes of the arrays for debugging
            print("dist1 size:", len(dist1))
            print("dist2 size:", len(dist2))
            print("dist3 size:", len(dist3))
            print("macd_values size:", len(macd_values))
            print("signal_values size:", len(signal_values))
            print("hist_values size:", len(hist_values))

            # Check if all indicators have sufficient historical data
            if len(dist1) < self.p.lookback or len(dist2) < self.p.lookback or len(dist3) < self.p.lookback:
                return  # Skip the current iteration if data is insufficient

            # Repeat the bollinger_signal to match the size of other inputs
            bollinger_signal = np.repeat(bollinger_signal, len(macd_values))

            # Combine the data into the input format for the DL model
            input_data = np.column_stack((dist1, dist2, dist3, bollinger_signal, macd_values, signal_values, hist_values))
            print("input_data shape:", input_data.shape)

            # Assuming you have a pre-trained RNN model built with Keras
            # Make predictions using the RNN model without verbosity
            predictions = self.model.predict(np.expand_dims(input_data, axis=0))[0]

            # Use the predictions to generate buy/sell/do-nothing signals
            if predictions[0] > self.p.buy_threshold:
                self.buy()
                self.signals.append('Buy')
            elif predictions[0] < -self.p.sell_threshold:
                self.sell()
                self.signals.append('Sell')
            else:
                self.signals.append('Do Nothing')

In [104]:
class DL_RNN_Strategy(bt.Strategy):
    params = (
        ('lookback', 20),  # Number of historical data points to use as input
        ('buy_threshold', 0),  # Threshold for buy signal (0 means buy when positive dist1)
        ('sell_threshold', 0),  # Threshold for sell signal (0 means sell when negative dist3)
    )

    def __init__(self, model):
        self.data_feed = self.datas[0]
        self.bollinger_signal = BollingerBands(self.data_feed)  # Calculate Bollinger Bands signal
        self.macd = MACD(self.data_feed)  # Calculate MACD using custom indicator
        self.model = model

    def next(self):
        if len(self.data) >= self.p.lookback:
            # Calculate Bollinger Bands signals and distances
            bollinger_signal = self.bollinger_signal.get(size=self.p.lookback)
            dist1 = self.bollinger_signal.lines.dist_upper[-self.p.lookback:]
            dist2 = self.bollinger_signal.lines.dist_lower[-self.p.lookback:]
            dist3 = self.bollinger_signal.lines.dist_upper[-self.p.lookback:]

            # Extract MACD values from the custom indicator
            macd_values = self.macd.macd.get(size=self.p.lookback)
            signal_values = self.macd.signal.get(size=self.p.lookback)
            hist_values = self.macd.hist.get(size=self.p.lookback)

            # Print the sizes of the arrays for debugging
            print("dist1 size:", len(dist1))
            print("dist2 size:", len(dist2))
            print("dist3 size:", len(dist3))
            print("macd_values size:", len(macd_values))
            print("signal_values size:", len(signal_values))
            print("hist_values size:", len(hist_values))

            # Check if all indicators have sufficient historical data
            if len(dist1) < self.p.lookback or len(dist2) < self.p.lookback or len(dist3) < self.p.lookback:
                return  # Skip the current iteration if data is insufficient

            # Combine the data into the input format for the DL model
            input_data = np.column_stack((dist1, dist2, dist3, bollinger_signal, macd_values, signal_values, hist_values))

            # Assuming you have a pre-trained RNN model built with Keras

            # Make predictions using the RNN model
            predictions = self.model.predict(np.expand_dims(input_data, axis=0))[0]

            # Use the predictions to generate buy/sell/do-nothing signals
            if predictions[0] > self.p.buy_threshold:
                self.buy()
            elif predictions[0] < -self.p.sell_threshold:
                self.sell()
            else:
                pass

In [105]:
data_feed = bt.feeds.PandasData(dataname=df)

cerebro = bt.Cerebro()
cerebro.adddata(data_feed)
cerebro.addstrategy(DL_RNN_Strategy, model=model)
cerebro.run() 

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [103]:
df

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-01-03 09:30:00-05:00,130.279999,130.899994,125.230003,125.459999,125.459999,27775505
2023-01-03 10:30:00-05:00,125.459999,125.870003,124.730003,125.345001,125.345001,18104999
2023-01-03 11:30:00-05:00,125.350197,125.370003,124.349098,124.660004,124.660004,11248777
2023-01-03 12:30:00-05:00,124.660004,124.940002,124.190002,124.649902,124.649902,8860080
2023-01-03 13:30:00-05:00,124.669899,125.000000,124.190002,124.570000,124.570000,8388062
...,...,...,...,...,...,...
2023-07-31 11:30:00-04:00,196.190002,196.369995,195.770004,196.154495,196.154495,3557921
2023-07-31 12:30:00-04:00,196.149994,196.250000,195.779999,196.129898,196.129898,2730653
2023-07-31 13:30:00-04:00,196.125000,196.389999,196.070007,196.160004,196.160004,2806909
